In [1]:
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")
library("DESeq2")
getwd()

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain intro

[1] "/home/gstupp/projects/Wolan/wolan/Fusion_AW_20170831"

In [2]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable$biological=factor(sampleTable$biological)
sampleTable$technical=factor(sampleTable$technical)
sampleTable$sample_type=factor(sampleTable$sample_type)
sampleTable$sample_type2=factor(sampleTable$sample_type2)
sampleTable$sample_type <- relevel(sampleTable$sample_type, ref="unenriched")
sampleTable$sample_type2 <- relevel(sampleTable$sample_type2, ref="unenriched")
sampleTable

,biological,technical,unenriched,probe,sample_type,sample_type2,sample_name,path
Neg1rep1,1,1,FALSE,FALSE,neg,probe,neg1,data/Neg1rep1/DTASelect-filter.txt
unenriched1,1,1,TRUE,FALSE,unenriched,unenriched,unenriched1,data/unenriched1/DTASelect-filter.txt
unenriched2,2,1,TRUE,FALSE,unenriched,unenriched,unenriched2,data/unenriched2/DTASelect-filter.txt
unenriched3,3,1,TRUE,FALSE,unenriched,unenriched,unenriched3,data/unenriched3/DTASelect-filter.txt
VK1rep1,1,1,FALSE,TRUE,probe,probe,vk1,data/VK1rep1/DTASelect-filter.txt
VK3rep1,1,1,FALSE,TRUE,probe,probe,vk3,data/VK3rep1/DTASelect-filter.txt
VK5rep1,1,1,FALSE,TRUE,probe,probe,vk5,data/VK5rep1/DTASelect-filter.txt


In [4]:
countData = read.csv("out/df_subtract.csv", row.names = 1)
locusTable = read.csv("out/locus_df.csv", row.names = 1)
sampleTable = sampleTable[colnames(countData),]

In [6]:
dds = DESeqDataSetFromMatrix(countData, colData = sampleTable, ~ sample_type)
dds = DESeq(dds)

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [7]:
head(counts(dds))

,VK1rep1,VK3rep1,VK5rep1,unenriched1,unenriched2,unenriched3
86769,0,6,0,12,13,13
187020,0,0,0,3,4,5
202956,11,10,18,14,15,12
209005,17,12,9,13,16,23
374028,0,0,0,16,16,16
566971,5,0,0,4,0,2


In [8]:
sampleTable

,biological,technical,unenriched,probe,sample_type,sample_type2,sample_name,path
VK1rep1,1,1,FALSE,TRUE,probe,probe,vk1,data/VK1rep1/DTASelect-filter.txt
VK3rep1,1,1,FALSE,TRUE,probe,probe,vk3,data/VK3rep1/DTASelect-filter.txt
VK5rep1,1,1,FALSE,TRUE,probe,probe,vk5,data/VK5rep1/DTASelect-filter.txt
unenriched1,1,1,TRUE,FALSE,unenriched,unenriched,unenriched1,data/unenriched1/DTASelect-filter.txt
unenriched2,2,1,TRUE,FALSE,unenriched,unenriched,unenriched2,data/unenriched2/DTASelect-filter.txt
unenriched3,3,1,TRUE,FALSE,unenriched,unenriched,unenriched3,data/unenriched3/DTASelect-filter.txt


In [9]:
#mcols(dds)<-cbind(mcols(dds),locusTable)
save(dds, file = "out/dds")

In [10]:
res = results(dds, contrast=c("sample_type","probe","unenriched"))
summary(res)


out of 14572 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2224, 15% 
LFC < 0 (down)   : 4046, 28% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [11]:
res = cbind(res, locusTable)
save(res, file = "out/res_probe_unenriched")

In [12]:
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/probe_unenriched_deseq_results.csv")